In [1]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import SubprocVecEnv
from gymnasium.envs.registration import register

from gymnasium import spaces
import torch as th
from torch import nn
import numpy as np
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

import gymnasium as gym
import os

from gym_data.envs import CarcassoneEnv
# device = torch.device("cuda")
register(id='Carcassone-v0',entry_point='gym_data.envs:CarcassoneEnv',) 

In [2]:
def linear_schedule(initial_value):
    """
    Linear learning rate schedule.
    :param initial_value: (float or str)
    :return: (function)
    """
    if isinstance(initial_value, str):
        initial_value = float(initial_value)

    def func(progress):
        """
        Progress will decrease from 1 (beginning) to 0
        :param progress: (float)
        :return: (float)
        """
        return progress * initial_value

    return func

In [3]:
iter = 0

env = make_vec_env(CarcassoneEnv, seed=1,  n_envs=16, vec_env_cls=SubprocVecEnv)

# model = A2C("MultiInputPolicy", env, verbose=1,tensorboard_log="runs/", device="auto", n_steps=5, learning_rate=linear_schedule(0.0001))
model = A2C.load("pretrained_carcassone3", env, verbose=1,tensorboard_log="runs/", device="auto", learning_rate=linear_schedule(0.0001))
import time
# model.set_parameters("model_carcassone")
start = time.time()
model.learn(total_timesteps=10_000_000)

end = time.time()

elapsed_time = end - start

print("Elapsed time: ", elapsed_time // 60, " minutes")

model.save("model_carcassone")
iter +=1

Total concat size:  2176
Logging to runs/A2C_14
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 101      |
|    ep_rew_mean        | -91.5    |
| time/                 |          |
|    fps                | 276      |
|    iterations         | 100      |
|    time_elapsed       | 740      |
|    total_timesteps    | 204800   |
| train/                |          |
|    entropy_loss       | -3.07    |
|    explained_variance | 0.000222 |
|    learning_rate      | 9.8e-05  |
|    n_updates          | 99       |
|    policy_loss        | 0.119    |
|    value_loss         | 131      |
------------------------------------


Process ForkServerProcess-6:
Process ForkServerProcess-1:
Process ForkServerProcess-15:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.11/site-packages/stable_baselines3/common/vec_env/subproc_vec_env.py", line 33, in _worker
    cmd, data = remote.recv()
                ^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
          ^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/multiprocessing/connection.py", line 430, in _recv_bytes
    buf = self._recv(4)
          ^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/multiprocessing/connection.py", line 395, in _recv
    chunk = read(handle, remaining)
            ^^^^^^^^^^^^^^^^^^

In [ ]:
model.save("model_carcassone")

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Mean Reward: {mean_reward}")